<a href="https://colab.research.google.com/github/LJ1201/MSDS-490-Bank_Campaign/blob/main/BankMkting_featureengineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import packages

In [431]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Import dataset

Data Source: http://archive.ics.uci.edu/ml/datasets/Bank+Marketing#

[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

Data Set Information:

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

There are four datasets:
1) bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]
2) bank-additional.csv with 10% of the examples (4119), randomly selected from 1), and 20 inputs.
3) bank-full.csv with all examples and 17 inputs, ordered by date (older version of this dataset with less inputs).
4) bank.csv with 10% of the examples and 17 inputs, randomly selected from 3 (older version of this dataset with less inputs).
The smallest datasets are provided to test more computationally demanding machine learning algorithms (e.g., SVM).

The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).


##Attribute Information:

Input variables:

#Bank client data:

1 - age (numeric)

2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)

4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

5 - default: has credit in default? (categorical: 'no','yes','unknown')

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - loan: has personal loan? (categorical: 'no','yes','unknown')



Related with the last contact of the current campaign:

8 - contact: contact communication type (categorical: 'cellular','telephone')

9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')

11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

#Other attributes:

12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

14 - previous: number of contacts performed before this campaign and for this client (numeric)

15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)


17 - cons.price.idx: consumer price index - monthly indicator (numeric)


18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)


19 - euribor3m: euribor 3 month rate - daily indicator (numeric)


20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):


21 - y - has the client subscribed a term deposit? (binary: 'yes','no')




#Relevant Papers:

S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimaraes, Portugal, October, 2011. EUROSIS. [bank.zip]



In [388]:
url = 'https://raw.githubusercontent.com/LJ1201/MSDS-490-Bank_Campaign/main/bank-additional-full.csv'
data = pd.read_csv(url, sep=';')

##EDA

In [389]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

##Categroical Variables

Categorical variables needs to be transformed into a form the model would take in. One hot encoding is a common way to perform the task

In [390]:
objList = []

for i in data.dtypes.index :
    if i =='y':continue
    #print(" here is i .....", i , " ..... and here is the type", dt[i] )
    if data.dtypes[i] in (["object"]) : objList.append( i )


In [391]:
objList

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']

In [392]:
data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [393]:
data['education'].value_counts()

data_1 = data.copy()
data_1 = data_1[data_1['education'] != 'illiterate']

data_1['edu_new'] = data_1['education'].map({'basic.4y': 'pre_high', 
                                         'basic.9y': 'pre_high',
                                         'basic.6y': 'pre_high',
                                         'university.degree' : 'post_high',
                                         'professional.course':'post_high',
                                         'high.school' : 'high',
                                         'unknown' :'unknown' })

In [394]:
def map_job(row):
    if row['job'] in ['admin.', 'management', 'entrepreneur','self-employed']:
        cat = 'white_collar'
    elif row['job'] in ['housemaid', 'blue-collar', 'technician', 'services']:
        cat = 'blue_collar'
    elif row['job'] in ['student', 'retired', 'unemployed']:
        cat = 'not_working'
    else:
        cat = 'unknown'
    return cat

data_1['job_new'] = data_1.apply(map_job, axis=1)
data_1['job_new'].value_counts(dropna = False)

blue_collar     21017
white_collar    16217
not_working      3606
unknown           330
Name: job_new, dtype: int64

In [395]:
def map_month(row):
    if row['month'] in ['mar', 'apr', 'may']:
        cat = 'spring'
    elif row['month'] in ['jun', 'jul', 'aug']:
        cat = 'summer'
    elif row['month'] in ['sep', 'oct', 'nov']:
        cat = 'fall'
    else:
        cat = 'winter'
    return cat

data_1['season'] = data_1.apply(map_month, axis=1)
data_1['season'].value_counts(dropna = False)

summer    18660
spring    16942
fall       5386
winter      182
Name: season, dtype: int64

In [396]:
data_1.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'edu_new', 'job_new',
       'season'],
      dtype='object')

### Data Transformation - V1 (Agreed on Team Discussion)

In [397]:
#This transformed dataframe is based on group discussion on 7/23
# - keep all numerical features as they are
# - consolidate categorical features (education, job, season) and drop originial ones

df_tran_1 = data_1[['age', 'marital', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'edu_new', 'job_new', 'season']]

In [398]:
df_tran_1.columns

Index(['age', 'marital', 'default', 'housing', 'loan', 'contact',
       'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y', 'edu_new', 'job_new', 'season'],
      dtype='object')

In [399]:
df_tran_1.head().T

,0,1,2,3,4
age,56,57,37,40,56
marital,married,married,married,married,married
default,no,unknown,no,no,no
housing,no,no,yes,no,no
loan,no,no,no,no,yes
contact,telephone,telephone,telephone,telephone,telephone
day_of_week,mon,mon,mon,mon,mon
duration,261,149,226,151,307
campaign,1,1,1,1,1
pdays,999,999,999,999,999


In [400]:
df_tran_1.y.value_counts()

no     36534
yes     4636
Name: y, dtype: int64

In [401]:
df_tran_1.columns

Index(['age', 'marital', 'default', 'housing', 'loan', 'contact',
       'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y', 'edu_new', 'job_new', 'season'],
      dtype='object')

In [402]:
df_tran_1.head().T

,0,1,2,3,4
age,56,57,37,40,56
marital,married,married,married,married,married
default,no,unknown,no,no,no
housing,no,no,yes,no,no
loan,no,no,no,no,yes
contact,telephone,telephone,telephone,telephone,telephone
day_of_week,mon,mon,mon,mon,mon
duration,261,149,226,151,307
campaign,1,1,1,1,1
pdays,999,999,999,999,999


In [406]:
df_tran_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41170 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41170 non-null  int64  
 1   marital         41170 non-null  object 
 2   default         41170 non-null  object 
 3   housing         41170 non-null  object 
 4   loan            41170 non-null  object 
 5   contact         41170 non-null  object 
 6   day_of_week     41170 non-null  object 
 7   duration        41170 non-null  int64  
 8   campaign        41170 non-null  int64  
 9   pdays           41170 non-null  int64  
 10  previous        41170 non-null  int64  
 11  poutcome        41170 non-null  object 
 12  emp.var.rate    41170 non-null  float64
 13  cons.price.idx  41170 non-null  float64
 14  cons.conf.idx   41170 non-null  float64
 15  euribor3m       41170 non-null  float64
 16  nr.employed     41170 non-null  float64
 17  y               41170 non-null 

In [407]:
df_tran_1.dtypes.index

Index(['age', 'marital', 'default', 'housing', 'loan', 'contact',
       'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y', 'edu_new', 'job_new', 'season'],
      dtype='object')

In [408]:
objList = []

for i in df_tran_1.dtypes.index :
    if i =='y':continue
    #print(" here is i .....", i , " ..... and here is the type", dt[i] )
    if df_tran_1.dtypes[i] in (["object"]) : objList.append( i )

for i in objList :
    thePrefix = "z_" + i
    y = pd.get_dummies( df_tran_1[i], prefix=thePrefix, drop_first=True )   
    df_tran_1 = pd.concat( [df_tran_1, y], axis=1 )
    df_tran_1 = df_tran_1.drop( i, axis=1 )

In [409]:
objList

['marital',
 'default',
 'housing',
 'loan',
 'contact',
 'day_of_week',
 'poutcome',
 'edu_new',
 'job_new',
 'season']

In [410]:
df_tran_1.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y',
       'z_marital_married', 'z_marital_single', 'z_marital_unknown',
       'z_default_unknown', 'z_default_yes', 'z_housing_unknown',
       'z_housing_yes', 'z_loan_unknown', 'z_loan_yes', 'z_contact_telephone',
       'z_day_of_week_mon', 'z_day_of_week_thu', 'z_day_of_week_tue',
       'z_day_of_week_wed', 'z_poutcome_nonexistent', 'z_poutcome_success',
       'z_edu_new_post_high', 'z_edu_new_pre_high', 'z_edu_new_unknown',
       'z_job_new_not_working', 'z_job_new_unknown', 'z_job_new_white_collar',
       'z_season_spring', 'z_season_summer', 'z_season_winter'],
      dtype='object')

In [411]:
df_tran_1['y'] = df_tran_1.y.map(dict(yes=1, no=0))

In [412]:



X = df_tran_1.loc[:, df_tran_1.columns != 'y']
y = df_tran_1.loc[:, df_tran_1.columns == 'y']


os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

length of oversampled data is  51102
Number of no subscription in oversampled data 25551
Number of subscription 25551
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


In [413]:
data_final_vars=df_tran_1.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]



logreg = LogisticRegression(solver= 'liblinear')
rfe = RFE(logreg,n_features_to_select=20)
rfe = rfe.fit(os_data_X.values, os_data_y.values.ravel())

features = list(os_data_X.columns[rfe.support_])

print('total features:', os_data_X.shape[1])

print(rfe.support_)
print(features)
print(rfe.ranking_)

total features: 35
[False False False False False False False False False False  True  True
  True  True False  True  True  True  True  True  True  True  True  True
 False  True  True  True  True False  True  True  True False False]
['z_marital_married', 'z_marital_single', 'z_marital_unknown', 'z_default_unknown', 'z_housing_unknown', 'z_housing_yes', 'z_loan_unknown', 'z_loan_yes', 'z_contact_telephone', 'z_day_of_week_mon', 'z_day_of_week_thu', 'z_day_of_week_tue', 'z_day_of_week_wed', 'z_poutcome_success', 'z_edu_new_post_high', 'z_edu_new_pre_high', 'z_edu_new_unknown', 'z_job_new_unknown', 'z_job_new_white_collar', 'z_season_spring']
[16 13  7 14  4  6  8 10  3 12  1  1  1  1 15  1  1  1  1  1  1  1  1  1
  5  1  1  1  1  2  1  1  1  9 11]


##Fit Model

##Model 1

Logistic Regression with top 20 selected by Recursive Feature Elimination method

In [428]:
features

['z_marital_married',
 'z_marital_single',
 'z_default_unknown',
 'z_housing_yes',
 'z_loan_yes',
 'z_contact_telephone',
 'z_day_of_week_mon',
 'z_day_of_week_thu',
 'z_day_of_week_tue',
 'z_day_of_week_wed',
 'z_poutcome_success',
 'z_edu_new_pre_high',
 'z_edu_new_unknown',
 'z_job_new_unknown',
 'z_job_new_white_collar',
 'z_season_spring']

In [415]:
X = os_data_X[features]
y = os_data_y['y']

In [416]:
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.500747
         Iterations 7
                            Results: Logit
Model:                Logit              Pseudo R-squared:   0.278     
Dependent Variable:   y                  AIC:                51216.3236
Date:                 2022-07-30 01:24   BIC:                51384.3136
No. Observations:     51102              Log-Likelihood:     -25589.   
Df Model:             18                 LL-Null:            -35421.   
Df Residuals:         51083              LLR p-value:        0.0000    
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       7.0000                                           
-----------------------------------------------------------------------
                        Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
z_marital_married       0.6134      nan      nan    nan     nan

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: divide by zero encountered in true_divide
  return self.params / self.bse


In [417]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [418]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.85


In [434]:
cm_1 = confusion_matrix(y_test, y_pred)
print(cm_1)

[[6579 1053]
 [1232 6467]]


In [ ]:
>>> clf = SVC(random_state=0)
>>> clf.fit(X_train, y_train)
SVC(random_state=0)
>>> plot_confusion_matrix(clf, X_test, y_test)  
>>> plt.show()

In [420]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      7632
           1       0.86      0.84      0.85      7699

    accuracy                           0.85     15331
   macro avg       0.85      0.85      0.85     15331
weighted avg       0.85      0.85      0.85     15331



###Model 2

In [421]:
features_2 = features

features_2.remove('z_housing_unknown')

features_2.remove('z_loan_unknown')


In [422]:
X = os_data_X[features_2]
y = os_data_y['y']

logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())


Optimization terminated successfully.
         Current function value: 0.502041
         Iterations 7
                            Results: Logit
Model:                Logit              Pseudo R-squared:   0.276     
Dependent Variable:   y                  AIC:                51346.6371
Date:                 2022-07-30 01:24   BIC:                51505.7855
No. Observations:     51102              Log-Likelihood:     -25655.   
Df Model:             17                 LL-Null:            -35421.   
Df Residuals:         51084              LLR p-value:        0.0000    
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       7.0000                                           
-----------------------------------------------------------------------
                        Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
z_marital_married       0.6015   0.0238  25.2536 0.0000  0.5549

In [423]:
features.remove('z_marital_unknown')
features.remove('z_edu_new_post_high')

In [424]:
X = os_data_X[features]
y = os_data_y['y']

logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())


Optimization terminated successfully.
         Current function value: 0.502069
         Iterations 7
                            Results: Logit
Model:                Logit              Pseudo R-squared:   0.276     
Dependent Variable:   y                  AIC:                51345.5045
Date:                 2022-07-30 01:24   BIC:                51486.9698
No. Observations:     51102              Log-Likelihood:     -25657.   
Df Model:             15                 LL-Null:            -35421.   
Df Residuals:         51086              LLR p-value:        0.0000    
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       7.0000                                           
-----------------------------------------------------------------------
                        Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
z_marital_married       0.6032   0.0233  25.9133 0.0000  0.5576

In [425]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.3, random_state=21)
logreg_2 = LogisticRegression()
logreg_2.fit(X_train_2, y_train_2)


LogisticRegression()

In [426]:
y_pred_2 = logreg_2.predict(X_test_2)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg_2.score(X_test_2, y_test_2)))

Accuracy of logistic regression classifier on test set: 0.84


In [427]:
cm_2 = confusion_matrix(y_test_2, y_pred_2)
print(cm_2)
print(classification_report(y_test_2, y_pred_2))

[[6577 1132]
 [1292 6330]]
              precision    recall  f1-score   support

           0       0.84      0.85      0.84      7709
           1       0.85      0.83      0.84      7622

    accuracy                           0.84     15331
   macro avg       0.84      0.84      0.84     15331
weighted avg       0.84      0.84      0.84     15331

